In [2]:
import pandas as pnds
import openpyxl
import datetime
import pickle, os

work_date = datetime.datetime.now() - datetime.timedelta(10)
target_date = work_date.strftime('%Y%m%d')                  # 어제 날짜 포맷
dfdata_dir = f'./data/{target_date}/dfdata/'                # 읽어들일 dfdata디렉토리 './data/YYYYMMDD/dfdata/'

# 5. KICC로부터 생성한 신용카드 승인내역 dataframe
card_df_filename = 'df_kicc_history_' + target_date         # 파일 이름: df_kicc_history_YYYYMMDD

# dfdata_dir = './data/20220713/dfdata/'
# card_df_filename = 'df_kicc_history_20220713'

#   KICC 카드 승인내역 Dataframe loading
try:
    with open(dfdata_dir + card_df_filename, "rb") as file:
        card_history_df = pickle.load(file)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[edi_opera.py - Reading Data] <{datetime.datetime.now()}> pickle file-reading error ({card_df_filename}) ===> {e}\n'
        )
    raise(e)

# #   내일 날짜는 제외 - 추후 추가 처리 가능성
# date_str = (work_date + datetime.timedelta(1)).strftime("%Y-%m-%d")                         # 내일 날짜 포맷
# card_history_df = card_history_df[card_history_df['date'].str.contains(date_str, na=False) == False]

# 6. 각 카드 내역들을 EDI 엑셀 파일의 카드 분류에따라 재분류
# 6-1. '카드분류'(card code) 컬럼과 그 영문 단축명인 'Description' 컬럼 생성
card_history_df['카드분류'] = '분류'
card_history_df['Description'] = '영문명'     # 'LT', 'KEB', 'JCB', 'VISA', 'MASTER', 'SA', 'SS', 'SH', 'BC', 'KB', 'HD', 'NH', 'CITI'

# 'LT'
condition = (card_history_df['매입카드사'].str.startswith('롯데', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9244'
card_history_df.loc[collected_cards, 'Description'] = 'LT'

# 'KEB'
condition = (card_history_df['매입카드사'] == '하나구외환')  & (card_history_df['발급카드사'].str.startswith('하나', na=False) | card_history_df['발급카드사'].str.startswith('토스', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9243'
card_history_df.loc[collected_cards, 'Description'] = 'KEB'

# 'JCB'
condition = (card_history_df['발급카드사'].str.contains('제이씨비', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9235'
card_history_df.loc[collected_cards, 'Description'] = 'JCB'

# 'VISA'
condition = (card_history_df['발급카드사'].str.contains('해외비자', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9231'
card_history_df.loc[collected_cards, 'Description'] = 'VISA'

# 'MASTER'
condition = (card_history_df['발급카드사'].str.contains('해외마스타', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9233'
card_history_df.loc[collected_cards, 'Description'] = 'MASTER'

# 'SA'
condition = (card_history_df['발급카드사'].str.contains('해외아멕스', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9232'
card_history_df.loc[collected_cards, 'Description'] = 'SA'

# 'SS'
condition = (card_history_df['발급카드사'].str.startswith('삼성', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9239'
card_history_df.loc[collected_cards, 'Description'] = 'SS'

# 'SH'
condition = (card_history_df['매입카드사'].str.startswith('신한', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9242'
card_history_df.loc[collected_cards, 'Description'] = 'SH'

# 'BC'
condition = (card_history_df['매입카드사'].str.startswith('비씨', na=False))      # 'CITI'씨티카드와 중복되지만, 'CITI'는 마지막에 다시 셋팅
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9237'
card_history_df.loc[collected_cards, 'Description'] = 'BC'

# 'KB'
condition = (card_history_df['매입카드사'].str.startswith('국민', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9238'
card_history_df.loc[collected_cards, 'Description'] = 'KB'

# 'HD'
condition = (card_history_df['매입카드사'].str.startswith('현대', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9245'
card_history_df.loc[collected_cards, 'Description'] = 'HD'

# 'NH'
condition = (card_history_df['매입카드사'].str.startswith('NH', na=False))
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9240'
card_history_df.loc[collected_cards, 'Description'] = 'NH'

# 'CITI'
condition = (card_history_df['발급카드사'].str.startswith('씨티', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
collected_cards = card_history_df[condition].index.tolist()
card_history_df.loc[collected_cards, '카드분류'] = '9241'
card_history_df.loc[collected_cards, 'Description'] = 'CITI'

# 10. grouping된 dataframe을 기존 KICC 신용거래내역 엑셀파일에 추가로 저장
# 10-1. 카드별 건수와 합계 금액 dataframe 생성: pivot(또는 groupby)를 이용하여 grouping
grouped_df = pnds.pivot_table(card_history_df, index=['매입카드사', 'Description'], values=['금액'], aggfunc=['count', 'sum'], margins = True)
# grouped_df = pnds.pivot_table(card_history_df, index=['Description'], values=['금액'], aggfunc=['count', 'sum'], margins = True)

# 10-2. 멀티 컬럼이므로 읽기 쉽도록 컬럼명 변경
grouped_df.columns = ['갯수:건수', '합계:금액']

# 10-3. dataframe의 행 순서를 EDI 엑셀파일 카드 순서대로 변경
#       EDI 엑셀파일의 카드 순서를 나타내는 리스트와 이 리스트를 이용하는 정렬용 key 함수

# ordered_card = {'LT': 1, 'KEB': 2, 'JCB': 3, 'VISA': 4, 'MASTER': 5, 'SA': 6,
#                 'SS': 7, 'SH': 8, 'BC': 9, 'KB': 10, 'HD': 11, 'NH': 12, 'CITI': 13}

# def card_sort(series):
#     """ordered_card 리스트를 이용하는 정렬용 key함수
#     인수로 전달된 카드 영문 단축명에 따라서, opdered_card의 value(정렬용 값)을 series로 반환

#     Args:
#         series (str): 카드 영문 단축명 series('Description' 컬럼)

#     Returns:
#         series (int): 인수로 들어온 series와 매칭되는 정열용 값을 가진 series
#     """

#     return series.apply(lambda col: ordered_card.get(col, 100))

def card_sort(series) -> pnds.Series:
        """ordered_card 리스트를 이용하는 정렬용 key함수
        인수로 전달된 카드 영문 단축명에 따라서, opdered_card의 value(정렬용 값)을 series로 반환

        Args:
            series (str): 카드 영문 단축명 series('Description' 컬럼)

        Returns:
            series (int): 인수로 들어온 series와 매칭되는 정열용 값을 가진 series
        """

        ordered_card = {'LT': 1, 'KEB': 2, 'JCB': 3, 'VISA': 4, 'MASTER': 5, 'SA': 6,
                        'SS': 7, 'SH': 8, 'BC': 9, 'KB': 10, 'HD': 11, 'NH': 12, 'CITI': 13}
        
        return series.apply(lambda col: ordered_card.get(col, 100))

#   정렬 작업 전후로, 멀티 인텍스를 임시로 해제 및 복구
grouped_df.reset_index(inplace=True)                                    # 먼저, dataframe의 index를 리셋해서 멀티 인덱스 해제
grouped_df.sort_values(by='Description', key=card_sort, inplace=True)   # key함수를 이용해서 정렬
grouped_df.set_index(['매입카드사', 'Description'], inplace=True)       # 다시 원래 형태의 grouping 형태대로 index 셋팅
# grouped_df.set_index(['Description'], inplace=True)       # 다시 원래 형태의 grouping 형태대로 index 셋팅

grouped_df

# -------------------------------
# Dataframe과 excel file 저장 방식은 크게 2가지로 나눈다.
# 선택은 2번: 이번 내용에서는 다중 인덱스로 셋팅되어서... 1번 방식에 메리트가 크게 없다
# -------------------------------

## I. 데이터는 Dataframe으로 다루다가, 엑셀의 각 cell 모양을 자세하게 다룰때는 아래 처럼 openpyxl의 dataframe을 사용해서 조정
# # openpyxl dataframe module
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.styles.alignment import Alignment

# # 새로운 워크북/시트 생성
# wb_out = openpyxl.Workbook()
# ws_out = wb_out.active

# # openpyxl에서 Datframe을 이용해서 저장
# for row in dataframe_to_rows(grouped_df, index=True, header=True):
#     # print(row)
#     ws_out.append(row)

# # 컬럼의 width 셋팅
# ws_out.column_dimensions['A'].width = 13.6
# ws_out.column_dimensions['B'].width = 13.6
# ws_out.column_dimensions['C'].width = 13.6

# # index와 column 이름을 강조하는 'Pandas' style
# for cell in ws_out['A'] + ws_out[1]:
#     cell.style = 'Pandas'

# # for rng in ws_out['B:B']:
# #     rng.number_format = '#,### '

# # 'B'에서 'C'컬럼들의 숫자 포맷 셋팅
# for rng in ws_out['B:C']:
#     for cell in rng:
#         cell.number_format = '#,### '

# # 엑셀 저장
# wb_out.save('./excel_grouped_df.xlsx')

## II. Fataframe으로만 작업해서 excel로 저장할 때 아래 처럼.
# # 10-4. 기존의 KICC 카드거래내역 엑셀 파일에 추가 모드로 저장
# try:
#     df_filename = 'df_kicc_history_' + target_date + '.xlsx'            # 저장파일 'df_kicc_history_YYYYMMDD.xlsx
#     print(target_dir + df_filename)

#     with pnds.ExcelWriter(target_dir + df_filename, mode='a', engine='openpyxl') as writer:     # 기존 파일 추가 mode 셋팅
#         grouped_df.to_excel(writer, sheet_name='groupped_calc', index=True)
# except Exception as e:
#     with open('./error.log', 'a') as file:
#         file.write(
#             f'[edi_opera.py - Making Excel] <{datetime.datetime.now()}> Pandas.ExcelWriter error ({df_filename}) ===> {e}\n'
#         )
#     raise(e)

갯수:건수    합계:금액
매입카드사 Description                
하나구외환 KEB              2   155166
      MASTER           5   318421
삼성카드  SS               1    88825
신한카드  SH               1   153560
비씨카드  BC               3   214330
국민카드  KB               6   384899
현대카드  HD               1    11000
All                   19  1326201